In [87]:
import glob
import re
import pickle
from collections import defaultdict

In [142]:
#Axcell imports
from pathlib import Path
from axcell.data.paper_collection import PaperCollection
from axcell.models.structure import TableType, TableStructurePredictor, TableTypePredictor

## Global vars

In [143]:
ROOT_PATH = Path('data')
MODELS_PATH = Path('models')

SOURCES_PATH = ROOT_PATH / 'sources'
PAPERS_PATH = ROOT_PATH / 'papers'

In [ ]:
pc = PaperCollection.from_files(PAPERS_PATH)

In [145]:
table_labels = {TableType.SOTA: 'leaderboard', TableType.ABLATION: 'ablation', TableType.IRRELEVANT: 'irrelevant'}

In [146]:
models_path = Path(MODELS_PATH)
ttp = TableTypePredictor(models_path, "table-type-classifier.pth")

/home/singh_shruti/anaconda3/envs/axcell/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.loss.BCEWithLogitsLoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/singh_shruti/anaconda3/envs/axcell/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'fastai.text.models.awd_lstm.AWD_LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/singh_shruti/anaconda3/envs/axcell/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has 

### Import collated data

In [26]:
with open("/home/singh_shruti/workspace/PaperAcceptancePrediction/shruti/features/iclr_arxiv_map.pkl", "rb") as f:
    iclr_arxiv_map = pickle.load(f)

In [25]:
with open("TABLE_Label_predicted.pkl", "rb") as f:
    table_label_info = pickle.load(f)

In [4]:
with open("leaderboard_table_refs.pkl", "rb") as some_file:
    leaderboard_table_refs = pickle.load(some_file)
with open("leaderboard_refs.pkl", "rb") as some_file:
    leaderboard_refs = pickle.load(some_file)

with open("new_leaderboard_refs.pkl", "rb") as some_file:
    NEW_leaderboard_refs = pickle.load(some_file)

### Check data format

In [5]:
list(NEW_leaderboard_refs.items())[0:3]

[('2017_B1-Hhnslg',
  {'refs': {'ldb': ["<ref id='bib-bib17'>2016</ref>",
     "<ref id='bib-bib1'>2013</ref>",
     "<ref id='bib-bib23'>2016</ref>",
     "<ref id='bib-bib29'>2016</ref>",
     "<ref id='bib-bib2'>2015</ref>",
     "<ref id='bib-bib22'>2017</ref>",
     "<ref id='bib-bib6'>2017</ref>"],
    'abl': []},
   'count': 12}),
 ('2017_B1-q5Pqxl', {'refs': {'ldb': []}, 'count': 0}),
 ('2017_B184E5qee',
  {'refs': {'ldb': ["<ref id='bib-bib37'>2012</ref>",
     "<ref id='bib-bib13'>2015</ref>",
     "<ref id='bib-bib36'>2016</ref>",
     "<ref id='bib-bib44'>2015</ref>",
     "<ref id='bib-bib40'>2014</ref>",
     "<ref id='bib-bib50'>2016</ref>",
     "<ref id='bib-bib49'>2014</ref>"]},
   'count': 10})]

In [17]:
list(leaderboard_table_refs.items())[0:3]

[('2017_B1-Hhnslg',
  {'tcount': [6, 4, 2, 0],
   'table_labels': [('leaderboard',),
    ('leaderboard',),
    ('leaderboard',),
    ('ablation',),
    ('leaderboard',),
    ('ablation',)]}),
 ('2017_B1-q5Pqxl',
  {'tcount': [2, 1, 0, 1],
   'table_labels': [('irrelevant',), ('leaderboard',)]}),
 ('2017_B16dGcqlx', {'tcount': [0, 0, 0, 0], 'table_labels': []})]

### Check if one is a subset of other

In [13]:
count = 0
reverse_count = 0
old_missing = 0
for k in NEW_leaderboard_refs:
    if "count" in NEW_leaderboard_refs[k]:
        if "count" in leaderboard_refs[k]:
            if NEW_leaderboard_refs[k]["count"] >= leaderboard_refs[k]["count"]:
                count +=1
            else:
                reverse_count += 1
        else:
            old_missing += 1
            #print("Count not present in old: ", leaderboard_refs[k])
    else:
        print("Count not present in NEW: ", NEW_leaderboard_refs[k])
print(count)
print(reverse_count)
print(old_missing)

530
0
158


### Hence, the dictionary to use from now id NEW_leaderboard_refs

### check missing ref stats

In [16]:
count = 0
for k in NEW_leaderboard_refs:
    if NEW_leaderboard_refs[k]["count"] == 0:
        count +=1
print(count, len(NEW_leaderboard_refs))
print(count/len(NEW_leaderboard_refs))

459 688
0.6671511627906976


In [24]:
count = 0

for k in leaderboard_table_refs:
    if "tcount" in leaderboard_table_refs:
        if k in NEW_leaderboard_refs:
            if "count" in NEW_leaderboard_refs[k]:
                if NEW_leaderboard_refs[k]["count"] == 0 and (leaderboard_table_refs[k]["tcount"][1]+leaderboard_table_refs[k]["tcount"][2])>0:
                    count += 1
            else:
                print("count absent in NEW: ", leaderboard_refs[k])
        else:
            print("Absent key: ", k)
    else:
        print("tcount absent: ", leaderboard_table_refs[k])

Absent key:  2017_B16dGcqlx
Absent key:  2017_B186cP9gx
Absent key:  2017_B1M8JF9xx
Absent key:  2017_B1TTpYKgx
Absent key:  2017_BJ3filKll
Absent key:  2017_BJ46w6Ule
Absent key:  2017_BJ5UeU9xx
Absent key:  2017_BJ6oOfqge
Absent key:  2017_BJAA4wKxg
Absent key:  2017_BJKYvt5lg
Absent key:  2017_BJO-BuT1g
Absent key:  2017_BJ_MGwqlg
Absent key:  2017_BJluGHcee
Absent key:  2017_BJmCKBqgl
Absent key:  2017_BJrFC6ceg
Absent key:  2017_BJxhLAuxg
Absent key:  2017_Bk0FWVcgx
Absent key:  2017_Bk8BvDqex
Absent key:  2017_BkLhzHtlg
Absent key:  2017_BkV4VS9ll
Absent key:  2017_BkVsEMYel
Absent key:  2017_Bkab5dqxe
Absent key:  2017_BkbY4psgg
Absent key:  2017_BkmM8Dceg
Absent key:  2017_Bks8cPcxe
Absent key:  2017_Bygq-H9eg
Absent key:  2017_Byk-VI9eg
Absent key:  2017_BymIbLKgl
Absent key:  2017_BysvGP5ee
Absent key:  2017_H1GEvHcee
Absent key:  2017_H1Heentlx
Absent key:  2017_H1W1UN9gg
Absent key:  2017_H1eLE8qlx
Absent key:  2017_HJWHIKqgl
Absent key:  2017_HJrDIpiee
Absent key:  2017_Hk

In [32]:
count

459

In [31]:
len(leaderboard_table_refs), len(NEW_leaderboard_refs)

(963, 688)

### Escape above errors

In [40]:
iclr_arxiv_map["2018_HyDAQl-AW"]

{'id': '2018_HyDAQl-AW',
 'title': 'Time Limits in Reinforcement Learning',
 'abstract': "In reinforcement learning, it is common to let an agent interact with its environment for a fixed amount of time before resetting the environment and repeating the process in a series of episodes. The task that the agent has to learn can either be to maximize its performance over (i) that fixed amount of time, or (ii) an indefinite period where the time limit is only used during training. In this paper, we investigate theoretically how time limits could effectively be handled in each of the two cases. In the first one, we argue that the terminations due to time limits are in fact part of the environment, and propose to include a notion of the remaining time as part of the agent's input. In the second case, the time limits are not part of the environment and are only used to facilitate learning. We argue that such terminations should not be treated as environmental ones and propose a method, specif

In [ ]:
2019_BJgqqsAct7, 2017_HyM25Mqel, 2018_HyDAQl-AW : no tables

In [ ]:
2019_BJxhijAcY7,  - table present in figure

In [ ]:
2019_BkgWHnR5tm(abl tables present of hyperparams), 2018_S1DWPP1A-, 2018_BJE-4xW0W(37 page paper with proofs and graph figs) : results in terms of graph figs

In [ ]:
2017_SJgWQPcxl(MNIST rejected in ICLR for same reason) - cannot find results

In [ ]:
2017_HJrDIpiee - some result in figure and most results are themselves ablation like 
metric is num_epochs_rqurd_to_reach_18_points_in_pong
# but since the contribution is primarily empirical rather than conceptual and algorithmic,
# there is a high bar for the rigorousness of the experiments. The reviewers generally did not find 
# the evaluation to be compelling enough in this regard.

# Just do it

In [54]:
# Check if count field is present in all
for k in NEW_leaderboard_refs:
    if not "count" in NEW_leaderboard_refs[k]:
        print(k)

print(list(NEW_leaderboard_refs.items())[0:2])

[('2017_B1-Hhnslg', {'refs': {'ldb': ["<ref id='bib-bib17'>2016</ref>", "<ref id='bib-bib1'>2013</ref>", "<ref id='bib-bib23'>2016</ref>", "<ref id='bib-bib29'>2016</ref>", "<ref id='bib-bib2'>2015</ref>", "<ref id='bib-bib22'>2017</ref>", "<ref id='bib-bib6'>2017</ref>"], 'abl': []}, 'count': 12}), ('2017_B1-q5Pqxl', {'refs': {'ldb': []}, 'count': 0})]


In [59]:
for k in leaderboard_table_refs:
    if not "tcount" in leaderboard_table_refs[k]:
        print(leaderboard_table_refs[k])

In [55]:
# Update correct count (there was some case where some ref was present but count was 0)
# print(list(NEW_leaderboard_refs.items())[0:2], end="\n\n")

for k in NEW_leaderboard_refs:
    if "refs" in NEW_leaderboard_refs[k]:
        refcount = 0
        if "ldb" in NEW_leaderboard_refs[k]["refs"]:
            refcount += len(NEW_leaderboard_refs[k]["refs"]["ldb"])
        if "abl" in NEW_leaderboard_refs[k]["refs"]:
            refcount += len(NEW_leaderboard_refs[k]["refs"]["abl"])
        
        if refcount > 0 and NEW_leaderboard_refs[k]["count"]==0:
            print(k)

In [62]:
count = 0
for k in leaderboard_table_refs:
    try:
        if (leaderboard_table_refs[k]["tcount"][1] + leaderboard_table_refs[k]["tcount"][2]) > 0:
            if k in NEW_leaderboard_refs and NEW_leaderboard_refs[k]["count"]==0:
                count += 1
    except Exception as ex:
        print(k)
        print(ex)
print(count)

459


In [63]:
count = 0
for k in leaderboard_table_refs:
    try:
        if (leaderboard_table_refs[k]["tcount"][1] + leaderboard_table_refs[k]["tcount"][2]) > 0:
            if k in NEW_leaderboard_refs and NEW_leaderboard_refs[k]["count"]>0:
                count += 1
    except Exception as ex:
        print(k)
        print(ex)
print(count)

229


In [64]:
229/len(leaderboard_table_refs)

0.23804573804573806

## Add from table caption

#### 1. check if any entry is missed by mistake

In [235]:
unextracted_papers = []
table_types_absent = []
new_table_counts = defaultdict(dict)
new_zero_table_counts = defaultdict(dict)

status_count = 0

In [236]:
for k, v in iclr_arxiv_map.items():
    if v["found"]:
        if status_count % 100 == 0:
            print("Status: ", status_count)

        try:
            path_split = v["arxivId"].split(".")

            # Extraction
            #extract(SOURCES_PATH / path_split[0] / v["arxivId"])

            paper = pc.get_by_id(v["arxivId"])

            if not paper:
                unextracted_papers.append(k)
                continue

            if len(paper.tables) > 0:                    
                # Add table type if absent
                if paper.tables[0].gold_tags == "":
                    if k in table_label_info:
                        for idx, t in enumerate(paper.tables):
                            t.gold_tags = table_label_info[k][idx]
                    else:
                        table_types_absent.append(k)
                        continue

                #Count the different types of tables
                table_type_count = [0, 0, 0]
                for tt in paper.tables:
                    if tt.gold_tags.lower().strip() == "leaderboard":
                        table_type_count[0] = table_type_count[0] + 1
                    elif tt.gold_tags.lower().strip() == "ablation":
                        table_type_count[1] = table_type_count[1] + 1
                    elif tt.gold_tags.lower().strip() == "irrelevant":
                        table_type_count[2] = table_type_count[2] + 1
                    else:
                        print("For k: {}, found unknown table type: {}".format(k, tt.gold_tags))

                # Save table stat in leaderboard_table_refs
                new_table_counts[k]["tcount"] = [len(paper.tables), table_type_count[0], table_type_count[1], table_type_count[2]]
            else:
                new_zero_table_counts[k]["tcount"] = [0, 0, 0, 0]
            
            if k in new_table_counts:
                new_table_counts[k]["table_labels"] = []
                for tablerol in paper.tables:
                    new_table_counts[k]["table_labels"].append((tablerol.gold_tags,))
                status_count += 1
        except Exception as ex:
            print(k, ex)

Status:  0
Status:  100
Status:  200
For k: 2018_ByJWeR1AW, found unknown table type: 
For k: 2018_ByJWeR1AW, found unknown table type: 
Status:  300
Status:  400
Status:  500
Status:  500
Status:  600
Status:  700
Status:  800
Status:  900
Status:  900
Status:  1000
Status:  1100
Status:  1200
Status:  1300
Status:  1400
For k: 2020_H1eqOnNYDH, found unknown table type: 
For k: 2020_H1eqOnNYDH, found unknown table type: 
Status:  1500
Status:  1600
Status:  1700
Status:  1800
Status:  1900


In [237]:
print(len(list(new_table_counts.keys())), len(new_zero_table_counts), len(unextracted_papers), len(table_types_absent))

1988 387 129 0


In [238]:
1988+387+129 #Is equal to the number of papers in iclr_arxiv_source_map found with arxiv source i.e. 2504

2504

#### 1.1 Inspect overwhelming number of table type absent

In [127]:
table_types_absent[0:4]

['2017_B1s6xvqlx', '2019_H1x-x309tm', '2019_H1x1noAqKX', '2019_H1xEwsR9FX']

In [130]:
print(iclr_arxiv_map["2019_H1x-x309tm"]["arxivId"])
paper = pc.get_by_id("1808.02941v2")
!ls data/htmls/1808/1808.02941v2/
!ls data/papers/1808/1808.02941v2/
print(len(paper.tables))
print(paper.tables[0].caption)
paper.tables[0]

1808.02941v2
index.html
layout_01.csv  metadata.json  table_01.csv  text.json
1
Table 1: Variants of generalized Adam


"^vtβ1,t","β1,t=0","β1,t≤β1,t−1 β1,t−−−→t→∞b≥0","β1,t=β1"
^vt=1,SGD,N/A∗,Heavy-ball method
^vt=1t∑ti=1g2i,AdaGrad,AdaFom,AdaFom
"vt=β2vt−1+(1−β2)g2t, ^vt=max(^vt−1,vt)",AMSGrad,AMSGrad,AMSGrad
^vt=β2^vt−1+(1−β2)g2t,RMSProp,N/A,Adam


In [132]:
print(iclr_arxiv_map["2017_B1s6xvqlx"]["arxivId"])
paper = pc.get_by_id("1704.02254v2")
!ls data/htmls/1704/1704.02254v2/
!ls data/papers/1704/1704.02254v2/
print(len(paper.tables))
print(paper.tables[0].caption)
paper.tables[0]

1704.02254v2
index.html
layout_01.csv  metadata.json  table_01.csv  text.json
1
(a) Graphical model representing (a) the recurrent structure used in Oh et al. (2015) and (b) our recurrent structure. Filled and empty nodes indicate observed and hidden variables respectively.


Game Name,DQN Score
Bowling,51.84
Breakout,396.25
Fishing Derby,19.30
Freeway,33.38
Ms Pacman,2963.31
Pong,20.88
Qbert,"14,865.43"
Riverraid,"13,593.49"
Seaquest,"17,250.31"
Space Invaders,2952.09


#### 1.2 Add table type for missed papers 

In [148]:
# tables_types = ttp.predict(paper, paper.tables)
#         for table, table_type in zip(paper.tables, tables_types):
#             table.gold_tags = table_labels[table_type]
#             table_info[k].append(table.gold_tags)
zero_tables = []
table_info = {}
# save_new_labels = {}

diff_labels = ["leaderboard", "ablation", "irrelevant"]

for k in table_types_absent:
    arxivid = iclr_arxiv_map[k]["arxivId"]
    paper = pc.get_by_id(arxivid)
    
    if len(paper.tables) > 0:
        if paper.tables[0].gold_tags.strip() == "":
            table_info[k] = []
            
            tables_types = ttp.predict(paper, paper.tables)
            
            for table, table_type in zip(paper.tables, tables_types):
                table.gold_tags = table_labels[table_type]
                table_info[k].append(table.gold_tags)
                
        elif paper.tables[0].gold_tags.strip() in diff_labels:
            continue
        else:
            print("Something table label found for k:{} as: {}!".format(k, paper.tables[0].gold_tags))
    else:
        zero_tables.append(k)

In [247]:
iclr_arxiv_map["2018_ByJWeR1AW"]["arxivId"]

'1806.03852v1'

In [248]:
paper = pc.get_by_id("1806.03852v1")
print(len(paper.tables))
tables_types = ttp.predict(paper, paper.tables)
print(tables_types)

9


[<TableType.ABLATION: 1>, <TableType.ABLATION: 1>, <TableType.IRRELEVANT: 2>, <TableType.IRRELEVANT: 2>, <TableType.ABLATION: 1>, <TableType.ABLATION: 1>, <TableType.SOTA: 0>, <TableType.ABLATION: 1>, <TableType.IRRELEVANT: 2>]


In [174]:
len(zero_tables), len(table_info)

(0, 2162)

#### 1.3 UPdate section 1.1 again using new table label info

In [241]:
unextracted_papers = []
still_table_types_absent = []
new_table_counts = defaultdict(dict)
new_zero_counts = defaultdict(dict)

status_count = 0

In [242]:
for k, v in iclr_arxiv_map.items():
    if v["found"]:
        if status_count % 100 == 0:
            print("Status: ", status_count)

        try:
            path_split = v["arxivId"].split(".")

            # Extraction
            #extract(SOURCES_PATH / path_split[0] / v["arxivId"])

            paper = pc.get_by_id(v["arxivId"])

            if not paper:
                unextracted_papers.append(k)
                continue

            if len(paper.tables) > 0:                    
                # Add table type if absent
                if paper.tables[0].gold_tags == "":
                    if k in table_label_info:
                        for idx, t in enumerate(paper.tables):
                            t.gold_tags = table_label_info[k][idx]
                    else:
                        if k in table_info:
                            for idx, t in enumerate(paper.tables):
                                t.gold_tags = table_info[k][idx]
                        else:
                            still_table_types_absent.append(k)
                            continue

                #Count the different types of tables
                table_type_count = [0, 0, 0]
                for tt in paper.tables:
                    if tt.gold_tags.lower().strip() == "leaderboard":
                        table_type_count[0] = table_type_count[0] + 1
                    elif tt.gold_tags.lower().strip() == "ablation":
                        table_type_count[1] = table_type_count[1] + 1
                    elif tt.gold_tags.lower().strip() == "irrelevant":
                        table_type_count[2] = table_type_count[2] + 1
                    else:
                        print("For k: {}, found unknown table type: {}".format(k, tt.gold_tags))

                # Save table stat in leaderboard_table_refs
                new_table_counts[k]["tcount"] = [len(paper.tables), table_type_count[0], table_type_count[1], table_type_count[2]]
            else:
                new_zero_table_counts[k]["tcount"] = [0, 0, 0, 0]
            
            if k in new_table_counts:
                new_table_counts[k]["table_labels"] = []
                for tablerol in paper.tables:
                    new_table_counts[k]["table_labels"].append((tablerol.gold_tags,))
                status_count += 1
        except Exception as ex:
            print(k, ex)

Status:  0
Status:  100
Status:  200
For k: 2018_ByJWeR1AW, found unknown table type: 
For k: 2018_ByJWeR1AW, found unknown table type: 
Status:  300
Status:  400
Status:  500
Status:  500
Status:  600
Status:  700
Status:  800
Status:  900
Status:  900
Status:  1000
Status:  1100
Status:  1200
Status:  1300
Status:  1400
For k: 2020_H1eqOnNYDH, found unknown table type: 
For k: 2020_H1eqOnNYDH, found unknown table type: 
Status:  1500
Status:  1600
Status:  1700
Status:  1800
Status:  1900


In [243]:
print(len(list(new_table_counts.keys())), len(new_zero_table_counts.keys()), len(unextracted_papers), len(still_table_types_absent))

1988 387 129 0


In [244]:
1988+387+129

2504

#### 1.4 Save this new leaderboard_table_refs

In [292]:
with open("leaderboard_table_refs.pkl", "wb") as f:
    pickle.dump(new_table_counts, f)

In [245]:
print(len(new_table_counts))
list(new_table_counts.items())[0]

1988


('2017_B1-Hhnslg',
 {'tcount': [6, 4, 2, 0],
  'table_labels': [('leaderboard',),
   ('leaderboard',),
   ('leaderboard',),
   ('ablation',),
   ('leaderboard',),
   ('ablation',)]})

In [164]:
print(len(leaderboard_table_refs))
list(leaderboard_table_refs.items())[0]

1181


('2017_B1-Hhnslg',
 {'tcount': [6, 4, 2, 0],
  'table_labels': [('leaderboard',),
   ('leaderboard',),
   ('leaderboard',),
   ('ablation',),
   ('leaderboard',),
   ('ablation',)]})

#### 1.5 Collate with the old and save the new table labels from ttp

In [166]:
len(table_label_info), len(table_info) #old, new

(967, 1195)

In [169]:
list(table_label_info.items())[0:2], list(table_info.items())[0:2]

([('2017_B1-Hhnslg',
   ['leaderboard',
    'leaderboard',
    'leaderboard',
    'ablation',
    'leaderboard',
    'ablation']),
  ('2017_B1-q5Pqxl', ['irrelevant', 'leaderboard'])],
 [('2017_B1s6xvqlx', ['irrelevant']), ('2019_H1x-x309tm', ['irrelevant'])])

In [194]:
collated_table_labels = table_info

for k in table_label_info:
    if k in collated_table_labels:
        if len(collated_table_labels[k])==len(table_label_info[k]) and collated_table_labels[k] == table_label_info[k]:
            continue
        else:
            print(k)
    else:
        collated_table_labels[k] = table_label_info[k]

In [246]:
nonzero = 0
for k in collated_table_labels:
    if collated_table_labels[k]:
        nonzero +=1
print(len(collated_table_labels), nonzero)

2162 1985


In [250]:
set(list(new_table_counts.keys()))-set(list(collated_table_labels.keys()))

{'2019_SJz6MnC5YQ', '2019_SyehMhC9Y7', '2019_rklwwo05Ym'}

In [254]:
new_table_counts["2019_rklwwo05Ym"]

{'tcount': [5, 1, 4, 0],
 'table_labels': [('ablation',),
  ('ablation',),
  ('ablation',),
  ('ablation',),
  ('leaderboard',)]}

In [256]:
for k in ['2019_SJz6MnC5YQ', '2019_SyehMhC9Y7', '2019_rklwwo05Ym']:
    collated_table_labels[k] = []
    for l in new_table_counts[k]["table_labels"]:
        collated_table_labels[k].append(l[0])

In [259]:
nonzero = 0
for k in collated_table_labels:
    if collated_table_labels[k]:
        nonzero +=1
print(len(collated_table_labels), nonzero)
# NOte that there are some 0 table entries in collated_table_labels

2165 1988


In [260]:
with open("TABLE_Label_predicted.pkl", "wb") as f:
    pickle.dump(collated_table_labels, f)

#### 2. Find already extracted ref from table cells and what is left

In [293]:
with open("leaderboard_table_refs.pkl", "rb") as some_file:
    leaderboard_table_refs = pickle.load(some_file)
print(len(leaderboard_table_refs))

1988


In [298]:
success = 0
refs_0 = 0
count_0 = 0
null_0 = 0

for k in leaderboard_table_refs:
    if k in NEW_leaderboard_refs:
        if "count" in NEW_leaderboard_refs[k]:
            if NEW_leaderboard_refs[k]["count"] > 0:
                success += 1
            else:
                refs_0 += 1
        else:
            count_0 +=1
    else:
        null_0 +=1

In [299]:
success, refs_0, count_0, null0

(229, 459, 0, 1300)

In [323]:
tot = 0 #(should end in 459 or more probably now after 2.1)
new_counts = 0

for k in leaderboard_table_refs:
    if k in NEW_leaderboard_refs and "count" in NEW_leaderboard_refs[k]:
        if NEW_leaderboard_refs[k]["count"] == 0:
            tot += 1
            if "refs" in NEW_leaderboard_refs[k] and "abl" in NEW_leaderboard_refs[k]["refs"] and len(NEW_leaderboard_refs[k]["refs"]["abl"]) > 0:
                new_counts +=1
print(new_counts)
print(tot)

58
1299


In [324]:
null_0 = 0
for k in leaderboard_table_refs:
    if not k in NEW_leaderboard_refs:
        null_0 +=1
print(null_0) #should show some dec from org count of 2300 after running Sec 2.1

270


#### 2.1 Run the previous extraction (from table cells) for papers for which extraction failed earlier

In [317]:
status_count = 0
err_count = 0
unext_papers = []
errs_ids = []

In [318]:
for k in leaderboard_table_refs:
    if not k in NEW_leaderboard_refs:
        if True:
            v = iclr_arxiv_map[k]
            if status_count % 10 == 0:
                print("Status: ", status_count)
                with open("leaderboard_refs.pkl", "wb") as some_file:
                    pickle.dump(NEW_leaderboard_refs, some_file)
                    
            try:
                path_split = v["arxivId"].split(".")

                # Extraction
                #extract(SOURCES_PATH / path_split[0] / v["arxivId"])

                paper = pc.get_by_id(v["arxivId"])
                
                if not paper:
                    unext_papers.append((k, v["arxivId"]))
                    continue
                
                if len(paper.tables) > 0:                    
                    # Add table type if absent
                    if paper.tables[0].gold_tags == "":
                        if k in collated_table_labels:
                            for idx, t in enumerate(paper.tables):
                                t.gold_tags = collated_table_labels[k][idx]
                        else:
                            tables_types = ttp.predict(paper, paper.tables)
                            for table, table_type in zip(paper.tables, tables_types):
                                table.gold_tags = table_labels[table_type]
                    
#                     #Count the different types of tables
#                     table_type_count = [0, 0, 0]
#                     for tt in paper.tables:
#                         if tt.gold_tags.lower().strip() == "leaderboard":
#                             table_type_count[0] = table_type_count[0] + 1
#                         elif tt.gold_tags.lower().strip() == "ablation":
#                             table_type_count[1] = table_type_count[1] + 1
#                         elif tt.gold_tags.lower().strip() == "irrelevant":
#                             table_type_count[2] = table_type_count[2] + 1
#                         else:
#                             print("Unknown table type: ", tt.gold_tags)
                    
#                     # Save table stat in leaderboard_table_refs
#                     leaderboard_table_refs[k]["tcount"] = [len(paper.tables), table_type_count[0], table_type_count[1], table_type_count[2]]
#                 else:
#                     leaderboard_table_refs[k]["tcount"] = [0, 0, 0, 0]
                
#                 leaderboard_table_refs[k]["table_labels"] = []
#                 for tablerol in paper.tables:
#                     leaderboard_table_refs[k]["table_labels"].append((tablerol.gold_tags,))
                
                if (leaderboard_table_refs[k]["tcount"][1]+leaderboard_table_refs[k]["tcount"][2]) > 0:
                    NEW_leaderboard_refs[k]["refs"] = {}
                    NEW_leaderboard_refs[k]["count"] = 0
                
                # If sota/leaderboard tables present, extract references from them
                if leaderboard_table_refs[k]["tcount"][1] > 0:
                    NEW_leaderboard_refs[k]["refs"]["ldb"] = []
                    for iterid, table in enumerate(paper.tables):
                        idx = iterid + 1
                        if table.gold_tags == "leaderboard":
                            
                            table_dir_path = "/home/singh_shruti/workspace/axcell_ws/axcell/notebooks/data/papers/" + path_split[0] + "/" + v["arxivId"]
                            table_files = glob.glob(table_dir_path+"/*.csv")
                            table_files_stripped = [tfs.rsplit("/", 1)[1] for tfs in table_files]
                            
                            candidate_name = "table_" + "{:02d}".format(idx) + ".csv"
                            if candidate_name in table_files_stripped:
                                cannot_use_f = open(table_dir_path + "/" + candidate_name, "r")
                                for line in cannot_use_f:
                                    m = re.findall("<ref id=[0-9a-zA-Z'-]*>[a-zA-Z 0-9\.]*</ref>", line)
                                    for iim in m:
                                        NEW_leaderboard_refs[k]["refs"]["ldb"].append(iim)
                                        NEW_leaderboard_refs[k]["count"] += 1
                            else:
                                print("For {} cannot find leaderboard table file: {}".format(k, candidate_name))
                            
                    NEW_leaderboard_refs[k]["refs"]["ldb"] = list(set(NEW_leaderboard_refs[k]["refs"]["ldb"]))
                    
                if leaderboard_table_refs[k]["tcount"][2] > 0:
                    NEW_leaderboard_refs[k]["refs"]["abl"] = []
                    for iterid, table in enumerate(paper.tables):
                        idx = iterid + 1
                        if table.gold_tags == "ablation":
                            
                            table_dir_path = "/home/singh_shruti/workspace/axcell_ws/axcell/notebooks/data/papers/" + path_split[0] + "/" + v["arxivId"]
                            table_files = glob.glob(table_dir_path+"/*.csv")
                            table_files_stripped = [tfs.rsplit("/", 1)[1] for tfs in table_files]
                            
                            candidate_name = "table_" + "{:02d}".format(idx) + ".csv"
                            if candidate_name in table_files_stripped:
                                cannot_use_f = open(table_dir_path + "/" + candidate_name, "r")
                                for line in cannot_use_f:
                                    m = re.findall("<ref id=[0-9a-zA-Z'-]*>[a-zA-Z 0-9\.]*</ref>", line)
                                    for iim in m:
                                        NEW_leaderboard_refs[k]["refs"]["abl"].append(iim)
                            else:
                                print("For {} cannot find ablation table file: {}".format(k, candidate_name))
                            
                    NEW_leaderboard_refs[k]["refs"]["abl"] = list(set(NEW_leaderboard_refs[k]["refs"]["abl"]))
                            
                status_count += 1
            except Exception as ex:
                print(k, v["arxivId"])
                print("Error: ", ex)
                err_count += 1
                errs_ids.append((k, v["arxivId"]))

Status:  0
Status:  10
Status:  20
Status:  30
Status:  40
Status:  50
Status:  60
Status:  70
Status:  80
Status:  90
Status:  100
Status:  110
Status:  120
Status:  130
Status:  140
Status:  150
Status:  160
Status:  170
Status:  180
Status:  190
Status:  200
Status:  210
Status:  220
Status:  230
Status:  240
Status:  250
Status:  260
Status:  270
Status:  280
Status:  290
Status:  300
Status:  310
Status:  320
Status:  330
Status:  340
Status:  350
Status:  360
Status:  370
Status:  380
Status:  390
Status:  400
Status:  410
Status:  420
Status:  430
Status:  440
Status:  450
Status:  460
Status:  470
Status:  480
Status:  490
Status:  500
Status:  510
Status:  520
Status:  530
Status:  540
Status:  550
Status:  560
Status:  570
Status:  580
Status:  590
Status:  600
Status:  610
Status:  620
Status:  630
Status:  640
Status:  650
Status:  660
Status:  670
Status:  680
Status:  690
Status:  700
Status:  710
Status:  720
Status:  730
Status:  740
Status:  750
Status:  760
Status:  7

In [319]:
status_count, err_count, len(unext_papers), unext_papers[0:3], len(errs_ids), errs_ids[0:3]

(1300, 0, 0, [], 0, [])

In [315]:
print(iclr_arxiv_map["2020_Skl4mRNYDr"]["arxivId"])
!ls data/papers/1810/1810.065*

for k in iclr_arxiv_map:
    if iclr_arxiv_map[k]["found"] and iclr_arxiv_map[k]["arxivId"] == "1810.06544v1":
        print(k)
for k in iclr_arxiv_map:
    if iclr_arxiv_map[k]["found"] and iclr_arxiv_map[k]["arxivId"] == "1810.06544v1":
        print(k)

1810.06544v4
data/papers/1810/1810.06530v5:
layout_01.csv  layout_04.csv  table_01.csv  table_04.csv
layout_02.csv  layout_05.csv  table_02.csv  table_05.csv
layout_03.csv  metadata.json  table_03.csv  text.json

data/papers/1810/1810.06543v1:
layout_01.csv  layout_03.csv  metadata.json  table_02.csv  table_04.csv
layout_02.csv  layout_04.csv  table_01.csv   table_03.csv  text.json

data/papers/1810/1810.06544v1:
layout_01.csv  layout_03.csv  table_01.csv  table_03.csv
layout_02.csv  metadata.json  table_02.csv  text.json

data/papers/1810/1810.06546v2:
layout_01.csv  layout_09.csv  layout_17.csv  table_07.csv  table_15.csv
layout_02.csv  layout_10.csv  metadata.json  table_08.csv  table_16.csv
layout_03.csv  layout_11.csv  table_01.csv   table_09.csv  table_17.csv
layout_04.csv  layout_12.csv  table_02.csv   table_10.csv  text.json
layout_05.csv  layout_13.csv  table_03.csv   table_11.csv
layout_06.csv  layout_14.csv  table_04.csv   table_12.csv
layout_07.csv  layout_15.csv  table_05.

In [316]:
print(len(NEW_leaderboard_refs))

689


In [320]:
print(len(NEW_leaderboard_refs))

1719


In [326]:
success = 0
refs_0 = 0
count_0_and_noabl = 0
count_nil_but_abl = 0
null_0 = 0

for k in leaderboard_table_refs:
    if k in NEW_leaderboard_refs:
        if "count" in NEW_leaderboard_refs[k]:
            if NEW_leaderboard_refs[k]["count"] > 0:
                success += 1
            else:
                refs_0 += 1
        else:
            if "refs" in  NEW_leaderboard_refs[k] and "abl" in NEW_leaderboard_refs[k]["refs"] and len(NEW_leaderboard_refs[k]["refs"]["abl"]) > 0:
                count_nil_but_abl +=1
            else:
                count_0_and_noabl +=1
    else:
        null_0 +=1

In [327]:
success, refs_0, count_nil_but_abl, count_0_and_noabl, null_0

(419, 1299, 0, 0, 270)

#### 3. Find if a sample reference is extracted by the code

In [341]:
papers_to_ext_refs = []
suc = 0
not_enough_tables = 0

In [342]:
for k in leaderboard_table_refs:
    if  (leaderboard_table_refs[k]["tcount"][1] + leaderboard_table_refs[k]["tcount"][2]) > 0:
        if k in NEW_leaderboard_refs:
            if "count" in NEW_leaderboard_refs[k] and NEW_leaderboard_refs[k]["count"] > 0: 
                suc +=1
            else:
                if "refs" in NEW_leaderboard_refs[k] and "abl" in NEW_leaderboard_refs[k]["refs"] and len(NEW_leaderboard_refs[k]["refs"]["abl"]) > 0:
                    suc +=1 
                else:
                    papers_to_ext_refs.append(k)
        else:
            papers_to_ext_refs.append(k)
    else:
        not_enough_tables +=1

print(suc, len(papers_to_ext_refs), not_enough_tables)

477 1241 270


In [343]:
1241+270+477

1988

# Our final set target is 1241 to augment 477

In [382]:
bbl_absent = []
bbl_multiple = []
bbl_unique = []

exceptions_ids = []

paper_bbl_dict = {}

In [383]:
for k in papers_to_ext_refs:
    arxivid = iclr_arxiv_map[k]["arxivId"]
    
    dir_name = arxivid.split(".")[0]
    
    # Read the bbl file to extract references
    potential_bbl_files = glob.glob("./data/unpacked_sources/{}/{}/*.bbl".format(dir_name, arxivid))

    if len(potential_bbl_files) == 1:
        bbl_unique.append(k)
        
        try:
            with open(potential_bbl_files[0], "r") as f:
                lines = f.readlines()

            found_bib_format = False
            for idx, l in enumerate(lines):
                if not found_bib_format and l.startswith("\\bibitem["):
                    #rint("KEY: {} FORMAT: {}".format(k, (l+lines[idx+1]).replace("\n", " ") ))
                    found_bib_format = True
                    break
                if not found_bib_format and l.startswith("\\bibitem{"):
                    #rint("KEY: {} FORMAT: {}".format(k, (l+lines[idx+1]).replace("\n", " ") ))
                    found_bib_format = True
                    break

            if not found_bib_format:
                print("\n Unusal KEY: {} FORMAT: {}".format(k, " ".join(lines[20:21])))
        except Exception as ex:
            print(k)
            exceptions_ids.append(k)
            #print("EXception: ", ex)
        
    elif len(potential_bbl_files) == 0:
        #print("No bbl files found: ", k)
        bbl_absent.append(k)
        
    else:
        #print("Multiple bbl found: ", k)
        bbl_multiple.append(k)

2017_BJYwwY9ll

 Unusal KEY: 2018_rJg4YGWRb FORMAT:   \sortlist[entry]{nty/global/}


 Unusal KEY: 2019_ryE98iR5tm FORMAT:     \name{author}{3}{}{%

2019_ryggIs0cYQ

 Unusal KEY: 2020_HkgxW0EYDS FORMAT:   \sortlist[entry]{nyt/global/}


 Unusal KEY: 2020_r1glygHtDB FORMAT:     \name{author}{7}{}{%


 Unusal KEY: 2020_rJlUhhVYvS FORMAT:   \sortlist[entry]{nty/global/}


 Unusal KEY: 2020_rygvFyrKwH FORMAT:   \sortlist[entry]{anyt/global/}



In [380]:
len(papers_to_ext_refs), len(bbl_absent), len(bbl_multiple), len(bbl_unique), len(exceptions_ids)
#Unusual format = 6

(1241, 45, 45, 1151, 2)

In [363]:
1151+45+45

1241

# Find refs of 1241 from table captions

In [418]:
table_caption_refs = defaultdict(list)

In [419]:
for k in papers_to_ext_refs:
    arxivid = iclr_arxiv_map[k]["arxivId"]
    paper = pc.get_by_id(arxivid)
    for idx, t in enumerate(paper.tables):
        if t.caption and t.caption.find(" et al") > -1:
            table_caption_refs[k].append((idx, t.caption))
        elif t.caption and t.caption.find(" & ") > -1:
            m = re.findall(" & [a-zA-Z 0-9\\.'~\{\}]*[ ]?\([1,2][0,1,2,8,9][0-9][0-9]\)", t.caption)
            if m:
#                 print(t.caption)
                table_caption_refs[k].append((idx, t.caption))

In [420]:
print(list(table_caption_refs.items())[0:2])
len(table_caption_refs)

[('2017_SJDaqqveg', [(1, 'Table 2: Our IWSLT 2014 machine translation results with a convolutional encoder compared to the previous work by Ranzato et al. Please see 1 for an explanation of abbreviations. The asterisk identifies results from (Ranzato et\xa0al., 2015). The numbers reported with ≤ were approximately read from Figure 6 of (Ranzato et\xa0al., 2015)')]), ('2017_SJc1hL5ee', [(6, 'Table 7: Comparison between CNNs and fastText with and without quantization. The numbers for\xa0Zhang et\xa0al. (2015) are reported from\xa0Xiao & Cho (2016). Note that for the CNNs, we report the size of the model under the assumption that they use float32 storage. For fastText(+PQ) we report the memory used in RAM at test time.')])]


45

In [421]:
unext_for_full_text = []
for k in papers_to_ext_refs:
    if not k in table_caption_refs.keys():
        unext_for_full_text.append(k)
len(unext_for_full_text)

1196

In [422]:
1196+45

1241

# Find refs of 1207 from full text

In [514]:
full_text_refs_dict = defaultdict(list)
full_text = {}
candidatesssssss = defaultdict(list)

In [516]:
for k in unext_for_full_text:
    arxivid = iclr_arxiv_map[k]["arxivId"]
    dir_name = arxivid.split(".")[0]
    
    paper = pc.get_by_id(arxivid)
    
    useful_tables = []
    for idx, l in enumerate(leaderboard_table_refs[k]["table_labels"]):
        if l[0] == "leaderboard" or l[0] == "ablation":
            useful_tables.append(idx)
    
    # Read all tex file to extract references
    full_latex_text = ""
    potential_tex_files = glob.glob("./data/unpacked_sources/{}/{}/*.tex".format(dir_name, arxivid))
    try:
        new_potents = []
        new_potents = glob.glob("./data/unpacked_sources/{}/{}/[a-zA-Z]*/*.tex".format(dir_name, arxivid))
    except Exception:
        continue
    
    
    potential_main_tex_files = []
    # Finding the main tex file
    for f in potential_tex_files:
        file_name = f.rsplit("/", 1)[1]
        if file_name.find("main") > -1:
            potential_main_tex_files.append(f)
        elif file_name.find("iclr") > -1:
            potential_main_tex_files.append(f)
    
    for f in new_potents:
        file_name = f.rsplit("/", 1)[1]
        if file_name.find("main") > -1:
            potential_main_tex_files.append(f)
        elif file_name.find("iclr") > -1:
            potential_main_tex_files.append(f)
    
    
    for f in potential_tex_files:
        try:
            with open(f, "r") as filehandler:
                file_lines = filehandler.readlines()
            full_latex_text += " ".join(file_lines)
        except Exception as ex:
            continue
    
    if new_potents:
        for f in new_potents:
            try:
                with open(f, "r") as filehandler:
                    file_lines = filehandler.readlines()
                full_latex_text += " ".join(file_lines)
            except Exception as ex:
                continue
#     try:
    full_text[k] = full_latex_text
    candidates_for_ref_methods = []
    for indices in useful_tables:
        row = paper.tables[indices].df.loc[0,:]
        col = paper.tables[indices].df.loc[:,0]

        for iii in row:
            if iii.raw_value.strip():
                candidates_for_ref_methods.append(iii.raw_value)
        for iii in col:
            if iii.raw_value.strip():
                candidates_for_ref_methods.append(iii.raw_value)
#     print(candidates_for_ref_methods, "\n\n\n")
    candidatesssssss[k] = candidates_for_ref_methods

    for candidate_ref_method in candidates_for_ref_methods:
        if candidate_ref_method.find("et al") > -1:
            full_text_refs_dict[k].append(candidate_ref_method)
        if full_latex_text.find(candidate_ref_method) > -1:
#             print(full_latex_text[full_latex_text.find(candidate_ref_method):full_latex_text.find(candidate_ref_method)+50])
            print(candidate_ref_method, k)
            for locss in re.finditer(candidate_ref_method, full_latex_text):
                start_loc = locss.start()
                end_loc = locss.end()
                chunk = full_latex_text[max(0, start_loc-50): min(end_loc+50, len(full_latex_text))]
                m = re.findall("cite[a-z\{]", chunk)
                if m:
                    full_text_refs_dict[k].append(chunk)
#     except Exception as ex:
#         print(ex)
#         continue
#     break

Exact Match 2017_B1-q5Pqxl
Exact Match 2017_B1-q5Pqxl
F1 2017_B1-q5Pqxl
F1 2017_B1-q5Pqxl
Random Guess 2017_B1-q5Pqxl
Logistic Regression 2017_B1-q5Pqxl
DCR 2017_B1-q5Pqxl
Match-LSTM with Ans-Ptr (Sequence) 2017_B1-q5Pqxl
Match-LSTM with Ans-Ptr (Boundary) 2017_B1-q5Pqxl
Match-LSTM with Ans-Ptr (Boundary+Search) 2017_B1-q5Pqxl
Match-LSTM with Ans-Ptr (Boundary+Search) 2017_B1-q5Pqxl
Match-LSTM with Ans-Ptr (Boundary+Search+b) 2017_B1-q5Pqxl
Match-LSTM with Bi-Ans-Ptr (Boundary+Search+b) 2017_B1-q5Pqxl
Match-LSTM with Ans-Ptr (Boundary+Search+en) 2017_B1-q5Pqxl
Our Model 2017_B1G9tvcgx
Domain 2017_B1GOWV5eg
FiGAR-TRPO 2017_B1GOWV5eg
TRPO 2017_B1GOWV5eg
Domain 2017_B1GOWV5eg
Ant 2017_B1GOWV5eg
Hopper 2017_B1GOWV5eg
Inverted Pendulum 2017_B1GOWV5eg
Inverted Double Pendulum 2017_B1GOWV5eg
Swimmer 2017_B1GOWV5eg
Variant 2017_B1GOWV5eg
Seaquest 2017_B1GOWV5eg
Space Invaders 2017_B1GOWV5eg
Asterix 2017_B1GOWV5eg
Variant 2017_B1GOWV5eg
FiGAR-50 2017_B1GOWV5eg
FiGAR-30-50 2017_B1GOWV5eg
FiGAR-P

Clean 2017_BJm4T4Kgx
Baseline 2017_BJm4T4Kgx
(standard training) 2017_BJm4T4Kgx
Adv. training 2017_BJm4T4Kgx
Deeper model 2017_BJm4T4Kgx
(standard training) 2017_BJm4T4Kgx
Deeper model 2017_BJm4T4Kgx
(Adv. training) 2017_BJm4T4Kgx
Adv. method 2017_BJm4T4Kgx
Training 2017_BJm4T4Kgx
Clean 2017_BJm4T4Kgx
Adv. method 2017_BJm4T4Kgx
Iter. l.l. 2017_BJm4T4Kgx
Iter. basic 2017_BJm4T4Kgx
Clean 2017_BJm4T4Kgx
No label leaking, 2017_BJm4T4Kgx
With label leaking, 2017_BJm4T4Kgx
training and eval using FGSM 2017_BJm4T4Kgx
FGSM 2017_BJm4T4Kgx
FGSM 2017_BJm4T4Kgx
FGSM 2017_BJm4T4Kgx
FGSM 2017_BJm4T4Kgx
basic iter. 2017_BJm4T4Kgx
basic iter. 2017_BJm4T4Kgx
basic iter. 2017_BJm4T4Kgx
basic iter. 2017_BJm4T4Kgx
iter l.l. 2017_BJm4T4Kgx
iter l.l. 2017_BJm4T4Kgx
iter l.l. 2017_BJm4T4Kgx
iter l.l. 2017_BJm4T4Kgx
top 1 2017_BJm4T4Kgx
top 5 2017_BJm4T4Kgx
Clean 2017_BJm4T4Kgx
No adversarial training 2017_BJm4T4Kgx
FGSM 2017_BJm4T4Kgx
Fast with predicted class 2017_BJm4T4Kgx
Fast entropy 2017_BJm4T4Kgx
Step 

error: nothing to repeat at position 0

In [507]:
len(full_text_refs_dict)

933

In [508]:
still_left_set = set(unext_for_full_text) - set(list(full_text_refs_dict.keys()))

In [509]:
len(still_left_set), list(still_left_set)[0:20]

(263,
 ['2019_SkVRTj0cYQ',
  '2019_rkevMnRqYQ',
  '2020_SJx0PAEFDS',
  '2020_SJlxglSFPB',
  '2020_Hkls_yBKDB',
  '2020_S1x2PCNKDB',
  '2020_S1xitgHtvS',
  '2017_SJJKxrsgl',
  '2017_B1YfAfcgl',
  '2018_rkpoTaxA-',
  '2017_BydARw9ex',
  '2017_ryelgY5eg',
  '2020_HJxiMAVtPH',
  '2018_rkr1UDeC-',
  '2019_SJequsCqKQ',
  '2020_HyeKcgHFvS',
  '2020_HkgtJRVFPS',
  '2017_SJNDWNOlg',
  '2018_HkanP0lRW',
  '2019_B1GSBsRcFX'])

In [511]:
# full_text["2020_ryxtCpNtDS"]
candidatesssssss["2019_B1GSBsRcFX"]

['Testing (training) accuracy (%)',
 'Testing (training) accuracy (%)',
 'Testing (training) accuracy (%)',
 'Testing (training) accuracy (%)',
 'Dataset',
 'MNIST',
 'SVHN',
 'CIFAR',
 'CIFAR',
 'CIFAR',
 'CIFAR+',
 'CIFAR+',
 'CIFAR+']

In [517]:
iclr_arxiv_map["2017_BydARw9ex"]["arxivId"]

'1611.09913v3'

In [518]:
ft = full_text["2017_BydARw9ex"]
#.find("Matching Networks")

In [522]:
for locss in re.finditer("\+RNN", ft):
    start_loc = locss.start()
    end_loc = locss.end()
    chunk = ft[max(0, start_loc-100): min(end_loc+1000, len(full_latex_text))]
    m = re.findall("cite[a-z\{]", chunk)
    if m:
        print(m, chunk)

['citep', 'citet'] itep{srivastava2015highway} and residual connections \citep{he2015deep}, respectively.  Indeed, the +RNN was inspired in 
 part by the coupled depth gate of \citet{srivastava2015highway}.
 
 \subsection{Recurrent Neural Network Architectures} \label{rnn architectures}
 Below we briefly define the RNN architectures used in this study.  Unless otherwise stated 
 $\WW$ denotes a matrix, $\bb$ denotes a vector of biases.  The symbol $\xxt$ is the 
 input at time $t$, and $\hht$ is the hidden state at time $t$.  Remaining vector 
 variables represent intermediate values.  The function  $\sigma(\cdot)$ denotes 
 the logistic sigmoid function and $\ft(\cdot)$ is either $\tanh$ or $\relu$, set 
 as a \hp (see Appendix, Section RNN \hps for the complete 
 list of \hps).  Initial conditions for the networks were set to a learned 
 bias. Finally, it is a well-known trick of the trade to initialize the gates of an 
 LSTM or GRU with a large bias to induce better gradient flow.  

In [396]:
list(leaderboard_table_refs.items())[0]

('2017_B1-Hhnslg',
 {'tcount': [6, 4, 2, 0],
  'table_labels': [('leaderboard',),
   ('leaderboard',),
   ('leaderboard',),
   ('ablation',),
   ('leaderboard',),
   ('ablation',)]})

In [ ]:
#         paper_bibitems = []
#         bibitem = []
#         start_looking = True
#         start_appending = False

#         for l in lines:
#             if start_looking and l.startswith("\\bibitem["):
#                 start_looking = False
#                 start_appending = True
#                 bibitem = [l.strip().replace("\n", " ")]
#                 continue
#             if start_appending:
#                 if l == "\n":
#                     paper_bibitems.append(bibitem)
#                     bibitem = []
#                     start_looking = True
#                     start_appending = False
#                 else:
#                     bibitem.append(l.strip().replace("\n", " "))

In [364]:
iclr_arxiv_map["2018_rJg4YGWRb"]["arxivId"]

'1803.03735v1'

In [368]:
#Error cannot open file to read
# !ls ./data/unpacked_sources/1803/1803.03735v1/*
# !cat ./data/unpacked_sources/1803/1803.03735v1/agnn.bbl

% $ biblatex auxiliary file $
% $ biblatex bbl format version 2.8 $
% Do not modify the above lines!
%
% This is an auxiliary file used by the 'biblatex' package.
% This file may safely be deleted. It will be recreated by
% biber as required.
%
\begingroup
\makeatletter
\@ifundefined{ver@biblatex.sty}
  {\@latex@error
     {Missing 'biblatex' package}
     {The bibliography requires the 'biblatex' package.}
      \aftergroup\endinput}
  {}
\endgroup


\refsection{0}
  \sortlist[entry]{nty/global/}
    \entry{BG17}{article}{}
      \name{author}{1}{}{%
        {{hash=9bd76342e74240fe5697674c12407f32}{%
           family={A.\bibnamedelimi Bojchevski},
           familyi={A\bibinitperiod\bibinitdelim B\bibinitperiod},
           given={S.\bibnamedelimi Gunnemann},
           giveni={S\bibinitperiod\bibinitdelim G\bibinitperiod}}}%
      }
      \strng{namehash}{9bd76342e74240fe5697674c12407f32}
      \strng{fullhash}{9bd76342e74240fe5697674c12407f32}
      \strng{authornamehash}{9bd76342e

In [ ]:
, 2020_HkgsWxrtPB

In [81]:
#Finding one paper with ref in table from its arxivid
for k in iclr_arxiv_map:
    if iclr_arxiv_map[k]["found"] and iclr_arxiv_map[k]["arxivId"].startswith("1702.08811"):
        print(k)

2017_SkB-_mcel


In [264]:
new_table_counts["2017_SkB-_mcel"]

{'tcount': [2, 2, 0, 0], 'table_labels': [('leaderboard',), ('leaderboard',)]}

In [283]:
for k in new_table_counts:
    try:
        if (new_table_counts[k]["tcount"][1] + new_table_counts[k]["tcount"][2]) > 0:
#             if k in NEW_leaderboard_refs and NEW_leaderboard_refs[k]["count"]==0:
#                 arxivid = iclr_arxiv_map[k]["arxivId"]
#                 paper = pc.get_by_id(arxivid)
#                 for t in paper.tables:
#                     if t.caption.find("bib") > -1:
#                         print(t.caption)
                    
#                 # check if reference is present in caption text
#                 break
            if not k in NEW_leaderboard_refs or (k in NEW_leaderboard_refs and NEW_leaderboard_refs[k]["count"]==0):
                arxivid = iclr_arxiv_map[k]["arxivId"]
                paper = pc.get_by_id(arxivid)
                for t in paper.tables:
                    if t.caption and t.caption.find(" et al") > -1:
#                     if t.caption and (t.caption.find("bib") > -1 or t.caption.find("et al") > -1):
                        print(k, arxivid)                    
#                 break
    except Exception as ex:
#         print(k)
        print(ex)

2017_SJDaqqveg 1607.07086v3
2019_BJgsN3R9Km 1910.01740v1
2019_BJgsN3R9Km 1910.01740v1
2019_BJgsN3R9Km 1910.01740v1
2019_ByxmXnA9FQ 1811.07308v4
2019_ByxmXnA9FQ 1811.07308v4
2019_ByxmXnA9FQ 1811.07308v4
2019_HJe62s09tX 1811.01124v3
2019_HkGmDsR9YQ 1810.00123v3
2019_HkGmDsR9YQ 1810.00123v3
2019_S1lVniC5Y7 1803.04439v2
2019_S1x2Fj0qKQ 1806.00420v2
2019_S1x2Fj0qKQ 1806.00420v2
2019_S1x2Fj0qKQ 1806.00420v2
2019_Sy4lojC9tm 1811.10959v3
2019_Sy4lojC9tm 1811.10959v3
2020_B1xq264YvH 1908.06938v2
2020_B1xq264YvH 1908.06938v2
2020_B1xq264YvH 1908.06938v2
2020_BJedHRVtPB 1906.06310v3
2020_BJedHRVtPB 1906.06310v3
2020_BJgza6VtPB 1811.02549v6
2020_BJgza6VtPB 1811.02549v6
2020_BJgza6VtPB 1811.02549v6
2020_BJgza6VtPB 1811.02549v6
2020_Bke8UR4FPB 1909.13488v1
2020_BkeaxAEKvB 1908.10396v2
2020_BkeaxAEKvB 1908.10396v2
2020_BkeaxAEKvB 1908.10396v2
2020_BkgXT24tDS 1909.13144v2
2020_Bkga90VKDB 1910.06720v1
2020_Bkgq9ANKvB 1910.03231v2
2020_Bkgq9ANKvB 1910.03231v2
2020_BklMDCVtvr 1910.09113v2
2020_Bklr0kBKvB

In [84]:
aid = iclr_arxiv_map["2017_SkB-_mcel"]["arxivId"]
print(aid)
paper = pc.get_by_id(aid)
print("Num tables: ", len(paper.tables))
for t in paper.tables:
    print(t.caption)

1702.08811v3
Num tables:  2
Table 1: Prediction accuracy ± standard deviation on the Amazon reviews dataset. The last two columns are taken directly from Louizos et al. (2016) and Ganin et al. (2016).
Table 2: Prediction accuracy ± standard deviation on the Office dataset. The first 10 rows are taken directly from the papers of Ganin et al. (2016) and Li et al. (2016). The models DLID –DANN are based on the AlexNet model, LSSA –AdaBN are based on the InceptionBN model, and our method (CMD) is based on the VGG16 model.


In [271]:
with open("./data/unpacked_sources/1702/1702.08811v3/final.bbl", "r") as f:
    lines = f.readlines()

In [278]:
paper_bibitems = []
bibitem = []
start_looking = True
start_appending = False

for l in lines:
    if start_looking and l.startswith("\\bibitem["):
        start_looking = False
        start_appending = True
        bibitem = [l.strip().replace("\n", " ")]
        continue
    if start_appending:
        if l == "\n":
            paper_bibitems.append(bibitem)
            bibitem = []
            start_looking = True
            start_appending = False
        else:
            bibitem.append(l.strip().replace("\n", " "))

In [274]:
lines

['\\begin{thebibliography}{34}\n',
 '\\providecommand{\\natexlab}[1]{#1}\n',
 '\\providecommand{\\url}[1]{\\texttt{#1}}\n',
 '\\expandafter\\ifx\\csname urlstyle\\endcsname\\relax\n',
 '  \\providecommand{\\doi}[1]{doi: #1}\\else\n',
 '  \\providecommand{\\doi}{doi: \\begingroup \\urlstyle{rm}\\Url}\\fi\n',
 '\n',
 '\\bibitem[Aljundi et~al.(2015)Aljundi, Emonet, Muselet, and\n',
 '  Sebban]{aljundi2015landmarks}\n',
 "Rahaf Aljundi, R{\\'e}mi Emonet, Damien Muselet, and Marc Sebban.\n",
 '\\newblock Landmarks-based kernelized subspace alignment for unsupervised domain\n',
 '  adaptation.\n',
 '\\newblock In \\emph{International Conference on Computer Vision and Pattern\n',
 '  Recognition}, pp.\\  56--63, 2015.\n',
 '\n',
 '\\bibitem[Ben-David et~al.(2010)Ben-David, Blitzer, Crammer, Kulesza, Pereira,\n',
 '  and Vaughan]{ben2010theory}\n',
 'Shai Ben-David, John Blitzer, Koby Crammer, Alex Kulesza, Fernando Pereira, and\n',
 '  Jennifer~Wortman Vaughan.\n',
 '\\newblock A theory of le

In [282]:
" ".join(paper_bibitems[0]).split("\\newblock")

["\\bibitem[Aljundi et~al.(2015)Aljundi, Emonet, Muselet, and Sebban]{aljundi2015landmarks} Rahaf Aljundi, R{\\'e}mi Emonet, Damien Muselet, and Marc Sebban. ",
 ' Landmarks-based kernelized subspace alignment for unsupervised domain adaptation. ',
 ' In \\emph{International Conference on Computer Vision and Pattern Recognition}, pp.\\  56--63, 2015.']

In [86]:
NEW_leaderboard_refs["1702.08811v3"], leaderboard_table_refs["1702.08811v3"]

({}, {})

In [73]:
dir(t)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_repr_html_',
 '_save_df',
 '_set_annotations',
 'caption',
 'dataset_text',
 'df',
 'display',
 'figure_id',
 'from_file',
 'gold_tags',
 'matrix',
 'matrix_gold_tags',
 'matrix_html',
 'matrix_layout',
 'matrix_tags',
 'name',
 'notes',
 'save',
 'set_layout',
 'set_tags',
 'shape',
 'sota_records']